# SQLAlchemy

SQLAlchemy - biblioteka skirta palengvinti darbą su duomenų bazėmis, komunikaciją Python <-> duomenų bazė. Ši biblioteka naudojama kaip ORM (angl. Object Relational Mapper) įrankis, kuris "išverčia" Python aprašytas klases į duomenų bazės lenteles ir automatiškai paverčia funkcijų kvietimus į SQL užklausas. SQLAlchemy biblioteka leidžia programuotojui dirbti neprisirišant prie konkrečios duomenų bazės, nes naudojama viena standartinė sąsaja darbui su daug skirtingų duomenų bazių. 

Kuriama programa <-> SQLAlchemy <-> DBAPI (angl. DataBase API) konkrečios duomenų bazės (MySQL, SQLite, PostgreSQL ir t.t.)

Prieš pradedant dirbti su SQLAlchemy ir duomenų bazėmis reikia sukurti varikliuką (angl. engine), kuris atliks komunikaciją su DBAPI funkcijomis. 

In [ ]:
# sukuriamas variklis darbui su SQLite duomenų baze
from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:", echo=True) # echo=True leis matyti visas SQL komandas

Norint nurodyti duomenų bazės lenteles ir jas atitinkančias Python klases naudojama deklaratyvi sistema (angl. Declarative system). Ši sistema leidžia susieti lenteles su klasėmis. Tam naudojama bazinė (angl. Base) klasė, kurios pagalba valdomas klasių ir lentelių sąsaja.

In [ ]:
# sukuriama deklaratyvi bazinė klasė
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

SQLAlchemy biblioteka palaiko visus populiariausius duomenų tipus. Minimalūs reikalavimai kuriant modelius: \_\_tablename\_\_ ir bent vienas stulpelis.

In [ ]:
# deklaratyvios klasės sukūrimas
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

Sukūrus User klasę per deklaratyvią sistemą, aprašėm informaciją apie lentelę, dar vadinamą metaduomenimis (angl. metadata). Sukurtas lentelės objektas yra didesnės kolekcijos MetaData narys. MetaData yra registras visų lentelių objektų ir jis atlieka jų sukūrimą duomenų bazėje.

In [ ]:
User.__table__

In [ ]:
# sukuriama lentelė duomenų bazėje
Base.metadata.create_all(engine)

Dabar Users klasė yra susieta su duomenų bazės lentele.

In [ ]:
# sukuriamas klasės Users objektas
tom_user = User(name="tom", fullname="Tomas Jonaitis", password="tomo_password")

In [ ]:
print(tom_user.name, tom_user.fullname, tom_user.password)

Sukūrus norimas lenteles galima pradėti "šnekėtis" (kurti, redaguoti, trinti, ieškoti įrašų) su duomenų baze. Tai atliekama per sisijas (angl. session).

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# duomenų perdavimas į sesiją
session.add(tom_user)

Kol kas duomenys nėra įrašyti į duomenų bazę, jie yra laukimo (angl. pending) būsenoje. Duomenys į lentelę bus surašyti tik iškvietus duomenų surašymo (angl. commit) komandą. Jei šiuo metu būtų iškviesta duomenų paieškos funkcija, tai laukimo būsenoje esantys duomenys būtų automatiškai surašyti į duomenų bazę. 

Biblioteka užtikrina, jog visos operacijos atliekamos sesijoje operuos tais pačiais duomenimis.

In [ ]:
# atliekama duomenų paieška duomenų bazėje pagal name lauką
our_user = session.query(User).filter_by(name="tom").first()

print(our_user)
print(tom_user is our_user)

In [ ]:
# į sesiją surašomi nauji User objektai add_all([])
session.add_all([
    User(name="jonas", fullname="Jonas Jonaitis", password="foobar"),
    User(name="petras", fullname="Petras Petraitis", password="xxg527"),
    User(name="zigmas", fullname="Zigmas Zigmaitis", password="blah")])

In [ ]:
# pakeičiamas vartotojo slaptažodis
tom_user.password = "f8s7ccs"

Surašytus duomenis į sesiją galima pamatyti:

In [ ]:
print(session.dirty)
print(session.new)
print(session.deleted)

In [ ]:
# inicijuojamas duomenų surašymas į duomenų bazė
session.commit()

In [ ]:
# duomenys surašyti duomenų bazėje, galima pamatyti sugeneruotas laukų reikšmes
print(tom_user.id)

# Duomenų atstatymas (angl. rolling back)

Kadangi sesijos dirba su tranzakcijomis, yra galimybė atstatyti neįrašytus (angl. commit) pakeitimus.

In [ ]:
tom_user.name = "vidmantas"

fake_user = User(name="netikras", fullname="Netikras vardas", password="Netikras slaptažodis")
session.add(fake_user)

In [ ]:
# duomenys yra surašyti į tranzakciją
print(session.query(User).filter(User.name.in_(["vidmantas", "netikras"])).all())

In [ ]:
# duomenų atstatymas
session.rollback()

In [ ]:
print(session.query(User).filter(User.name.in_(["vidmantas", "netikras", "tom"])).all())
print(fake_user in session)

# Užklausų formavimas

Užklausos (angl. query) objektas sukuriamas naudojant Session metodą query().

In [ ]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

In [ ]:
# konkrečių laukų užklausa atsakymą grąžina kaip sąrašą
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

for row in session.query(User.name, User.fullname):
    print(row)
    
for row in session.query(User.name, User.fullname):
    print(row.name, row.fullname)
    
for row in session.query(User.name.label("name_label")).all():
    print(row.name_label)

In [ ]:
# duomenų ribojimas LIMIT ir OFFSET
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

In [ ]:
# filtravimas su filter_by(). filter_by() skirtas paprastų užklausų formavimui atrenkant juos 
# pagal stulpelių pavadinimus, naudojant kwargs
for name in session.query(User.name).filter_by(name="tom"):
    print(name)
    
for name in session.query(User.name).filter_by(name="tom").filter_by(fullname="Tomas Jonaitis"):
    print(name)

In [ ]:
# filtravimas su filter(). filter() leidžia formuoti sudėtingesnes užklausas nenaudojant kwargs
for name in session.query(User.name).filter(User.fullname=="Tomas Jonaitis"):
    print(name)
    
for name in session.query(User.name).filter(User.fullname=="Tomas Jonaitis").filter(User.name=="tom"):
    print(name)

Dažniausiai naudojamos filtravimo operacijos:
1. Lygu (angl. equal)
```python
query.filter(User.name == "tom")
```
2. Nelygu (angl. not equal)
```python
query.filter(User.name != "tom")
```
3. Panašu (angl. like)
```python
query.filter(User.name.like("%tom%"))
```
4. Panašu skiriant didžiąsias ir mažąsias raides (angl. ilike)
```python
query.filter(User.name.ilike("%tom%"))
```
5. Yra, egzistuoja (angl. in)
```python
query.filter(User.name.in_(["tom", "ed"]))
# galima naudoti ir query() objektą
query.filter(User.name.in_(session.query(User.name).filter(User.name.like("%tom%"))))
```
6. Nėra, neegzistuoja (angl. not in)
```python
query.filter(~User.name.in_(["ed", "tom"]))
```
7. Yra tuščia (angl. is null)
```python
query.filter(User.name == None)
query.filter(User.name.is_(None))
```
8. Nėra tuščia (angl. is not null)
```python
query.filter(User.name != None)
query.filter(User.name.isnot(None))
```
9. Ir operatorius (angl. and)
```python
# naudojant and_()
from sqlalchemy import and_
query.filter(and_(User.name == "tom", User.fullname == "Tomas Jonaitis"))
# kelios užklausos viename filter()
query.filter(User.name == "tom", User.fullname == "Tomas Jonaitis")
# keli filter() arba filter_by() metodai
query.filter(User.name == "tom").filter(User.fullname == "Tomas Jonaitis")
```
10. Arba operatorius (angl. or)
```python
from sqlalchemy import or_
query.filter(or_(User.name == "tom", User.name == "mot"))
```
11. Atitinka (angl. match). Ši komanda priklausomai nuo naudojamos duomenų bazės yra verčiama arba į MATCH arba į CONTAINS funkcijas. Taip pat šios komandos veikimas priklausis nuo naudojamos duomenų bazės bibliotekos.
```python
query.filter(User.name.match("tom"))
```

# Sąrašų grąžinimas

Query() objektas turi daug metodų, kurie iškarto atlieka SQL užklausą ir grąžina rezultatus.

In [ ]:
query = session.query(User).filter(User.name.like("%m%")).order_by(User.id)

In [ ]:
# all() grąžina visus objektus kaip sąrašą
print(query.all())

In [ ]:
# first() grąžina pirmą objektą iš duomenų bazės
print(query.first())

In [ ]:
# one() pilnai parsiunčia visas užklausos eilutes, jei rezultatų nėra 1 
# iššaukiama klaida MultipleResultsFound arba NoResultFound
print(query.one())

In [ ]:
# kaip ir one(), tik jei nerasta jokių įrašų grina None ir nekviečia klaidos
print(query.one_or_none())

In [ ]:
# scalar() iškviečia one() metodą ir jei neįvyksta klaida grąžina pirmos eilutės pirmą stulpelį
print(query.scalar())

# Tekstinių užklausų formavimas

Tekstinių komandų naudojimas leidžia suformuoti lankstesnes užklausas.

In [ ]:
# teksto panaudojimas užklausų formavime
from sqlalchemy import text

for user in session.query(User).filter(text("id<224")).order_by(text("id")).all():
    print(user.name)

In [ ]:
# parametrų nurodymas su params komanda
session.query(User).filter(text("id<:value and name=:name")).params(value=224, name="tom").order_by(User.id).one()

In [ ]:
# naudojant text() ir from_statement() galima suformuoti pilną tekstinę SQL užklausą
session.query(User).from_statement(text("SELECT * FROM users where name=:name")).params(name="tom").all()

In [ ]:
# kai užklausoje yra dubliuoti pavadinimai yra sudėtinga analizuoti ir formuoti užklausas tam tikslui galima susieti 
# tekste aprašytus stulpelius su objekto laukais
stmt = text("SELECT name, id, fullname, password FROM users where name=:name")
stmt = stmt.columns(User.name, User.id, User.fullname, User.password)
session.query(User).from_statement(stmt).params(name="tom").all()

In [ ]:
# užklausos laukus galima nurodyti ir query() užklausoje
stmt = text("SELECT name, id FROM users where name=:name")
stmt = stmt.columns(User.name, User.id)
session.query(User.id, User.name).from_statement(stmt).params(name="tom").all()

# Įrašų skaičiavimas

In [ ]:
# skaičiuojant tokiu būdu visa duomenų užklausa įdedama į subužklausą SELECT count(*) AS count_1 FROM (SELECT .....)
session.query(User).filter(User.name.like("%m%")).count()

In [ ]:
# skaičiuojant naudojant func.count() suformuojama užklausa kur skaičiuojamas tik konkretus laukas
# SELECT count(users.name) AS count_1, users.name AS users_name
# FROM users GROUP BY users.name
from sqlalchemy import func
session.query(func.count(User.name), User.name).group_by(User.name).all()

In [ ]:
# paprastos skaičiuojamos užklausos 
# SELECT count(*) FROM table
session.query(func.count("*")).select_from(User).scalar()

In [ ]:
session.query(func.count(User.id)).scalar()

# Ryšių tarp lentelių kūrimas

Sukuriamas naujas modelis, kuris bus susietas su User modeliu vienas su daug ryšiu, t.y. vienas User įrašas turės daug adresų. ForeignKey - nurodo, jog reikšmės stulpelyje user_id yra susietos su kito modelio (lentelės) stulpelio reikšmėmis. Relationship - nurodo ORM, jog Address klasė turi būti sisieta su User klase ir atvirkščiai.

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = "addresses"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"))
    
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [ ]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

In [ ]:
# pakeitmai surašomi į duomenų bazę
Base.metadata.create_all(engine)

In [ ]:
# sukuriamas naujas User įrašas, jo adresus galima pasiekti per User.addresses
sim = User(name="simas", fullname="Simas Simavičius", password="gjffdd")
print(sim.addresses)

In [ ]:
# naujų adresų sukūrimas galimas per User objektą
sim.addresses = [
    Address(email_address="sim@google.com"),
    Address(email_address="sim@yahoo.com")
]

print(sim.addresses)

In [ ]:
# kadangi modeliai yra susieti į abi puses galima pasiekti bet kuriuos susietų modelių eleentus
print(sim.addresses[1].user)

In [ ]:
# duomenys surašomi į duomenų bazę
session.add(sim)
session.commit()

In [ ]:
# užklausiant vartotojo duomenų adresai nėra užklausiami ta pačia užklausa
sim = session.query(User).filter_by(name="simas").one()
print(sim)

In [ ]:
# vartotojo adresai yra užklausiami iš duomenų bazės tik tada kai jų reikia
print(sim.addresses)

# Susietų lentelių užklausos

Paprastų susietų lentelių užklausų formavimui galima naudoti Query.filter() atrintki kelių susietų lentelių duomenis.

In [ ]:
# FROM users, addresses WHERE users.id = addresses.user_id
for u, a in session.query(User, Address).filter(User.id==Address.user_id).\
                                        filter(Address.email_address=="sim@google.com").\
                                        all():
    print(u)
    print(a)

In [ ]:
# FROM users, addresses WHERE users.id = addresses.user_id
for u, a in session.query(User, Address).filter(User.id==Address.user_id).\
                                        filter(User.name=="simas").\
                                        all():
    print(u)
    print(a)

In [ ]:
# FROM users JOIN addresses ON users.id = addresses.user_id
session.query(User).join(Address).filter(Address.email_address=="sim@google.com").all()

In [ ]:
# FROM users JOIN addresses ON users.id = addresses.user_id
session.query(User).join(Address).filter(User.name.like("%m%")).all()

Kadangi tarp User ir Address lentelių yra tik vienas raktas (angl. foreign key) query.join() gali lengvai sujungti lenteles, tačiau jei raktų nebūtų arba būtų keli reikia naudoti kitą struktūrą.
```python
query.join(Address, User.id==Address.user_id)    # nurodoma kokius laukus jungti
query.join(User.addresses)                       # nurodomas ryšys iš kairės į dešinę
query.join(Address, User.addresses)              # nurodomas sujungimas per modelių ryšį
query.join("addresses")                          # tas pats naudojant tekstą
```

Išorinis sujungimas (angl. outer join):
```python
query.outerjoin(User.addresses)
```

### Lentelių pavadinimų santrumpos (angl. alias)

Formuojant užklausas kartais pasitaiko situacijų, kai ta pati lentelė turi būti iškviesta kelis kartus. Esant tokiai situacijai reikia naudoti lentelių santrumpas, kurios leidžia atskirti dvi tas pačias lenteles. 

In [ ]:
from sqlalchemy.orm import aliased
adalias1 = aliased(Address)
adalias2 = aliased(Address)

for username, email1, email2 in session.query(User.name, adalias1.email_address, adalias2.email_address).\
                                join(adalias1, User.addresses).\
                                join(adalias2, User.addresses).\
                                filter(adalias1.email_address=="sim@google.com").\
                                filter(adalias2.email_address=="sim@yahoo.com"):
    print(username, email1, email2)

### Užklausos iš užklausų (angl. subqueries)

Norint gauti informaciją apie tai kiek adresų turi kiekvienas vartotojas būtų formuojama SQL užklausa:
```SQL
SELECT users.*, adr_count.address_count FROM users LEFT OUTER JOIN
    (SELECT user_id, count(*) AS address_count
        FROM addresses GROUP BY user_id) AS adr_count
    ON users.id=adr_count.user_id
```

Formuojama užklausa, kuri suskaičiuoja adresų kiekius, o antroje užklausa suformuoja galutinį atsakymą.

In [ ]:
# suformuojama užklausa kuri suskaičiuoja adresų kiekius
from sqlalchemy.sql import func
stmt = session.query(Address.user_id, func.count("*").\
                     label("address_count")).\
                     group_by(Address.user_id).subquery()

In [ ]:
# turint užklausą kuri suskaičiuoja kiekius, ją galima traktuoti kaip lentelę ir naudoti kitose užklausose
# stulpeliai iš subužklausos pasiekiami per 'c' atributą
for u, count in session.query(User, stmt.c.address_count).outerjoin(stmt, User.id==stmt.c.user_id).order_by(User.id):
    print(u, count)

In [ ]:
# norint gauti adresų elementą naudojamas aliased()
stmt = session.query(Address).\
                    filter(Address.email_address != 'j25@yahoo.com').\
                    subquery()
        
adalias = aliased(Address, stmt)

for user, address in session.query(User, adalias).\
        join(adalias, User.addresses):
    print(user)
    print(address)

## Exists komandos naudojimas

Exists raktažodis SQL kalboje grąžina True jei pagal duotą išraišką egzistuoja eilučių.

In [ ]:
# exists komandos konstrukcija
from sqlalchemy.sql import exists

stmt = exists().where(Address.user_id==User.id)
for name, in session.query(User.name).filter(stmt):
    print(name)

In [ ]:
# exists komandos alternatyva - any()
for name, in session.query(User.name).\
        filter(User.addresses.any()):
    print(name)

In [ ]:
# any() komandoje galima nurodyti ir filtravimo kriterijus
for name, in session.query(User.name).\
        filter(User.addresses.any(Address.email_address.like("%google%"))):
    print(name)

In [ ]:
# has() komanda yra alternatyva any() tik naudojama daug su vienu ryšiuose
session.query(Address).filter(~Address.user.has(User.name=="sim")).all()

# Įrašų šalinimas

In [ ]:
# įrašų šalinimas
session.delete(sim)
session.query(User).filter_by(name="sim").count()

In [ ]:
# adresai lieka neištrinti
session.query(Address).filter(
    Address.email_address.in_(["sim@google.com", "sim@yahoo.com"])
).count()

Ištrinami tik User duomenys, tačiau susieti Address duomenys lieka ništrinti, o jų User laukai tampa NULL. Todėl norint jog ištrinant tėvinį įrašą būtų ištrinti ir susieti įrašai reikia SQLAlchemy nurodyti tai (angl. cascade).

In [ ]:
session.close()
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    addresses = relationship("Address", back_populates='user',
                             cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

    
sim = session.query(User).get(5)
print(sim)
del sim.addresses[1]

session.query(Address).filter(
    Address.email_address.in_(["sim@google.com", "sim@yahoo.com"])
).count()

In [ ]:
session.delete(sim)
session.query(User).filter_by(name="sim").count()

session.query(Address).filter(
    Address.email_address.in_(["sim@google.com", "sim@yahoo.com"])
).count()

# Užduotys

1.
2.
3.
4.
5.